# Laboratorio 4: Redes Neuronales
### Ricardo Valenzuela 18762
### Juan Diego Solorzano 18151

Este laboratorio se realizo basandonos en el ejemplo visto en clase, se realizaron los debidos cambios para responder las diferentes preguntas de los incisos del laboratorio.

## Codigo ejemplo visto en clase con cambios

In [89]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # Para graficar en 3-D

#### Generar datos al azar para entrenar al modelo

Trabajaremos con dos variables de entrada, las x1 y x2 en nuestros ejemplos anteriores. Se generan al azar a partir de una distribución uniforme.

Se creará una matriz con estas dos variables.  La matriz X del modelo lineal y = x * w + b

In [90]:
observaciones = 1000

x1 = np.random.uniform(low=-10, high=10, size=(observaciones,1))
x2 = np.random.uniform(-10, 10, (observaciones,1))

entradas = np.column_stack((x1,x2))

print (entradas.shape)

(1000, 2)


### Generar las metas a las que debemos apuntar

Inventaremos una función f(x1, x2) = 2 * x1 - 3 * x2 + 5 + <ruido pequeño>.  El ruido es para hacerlo más realista.

Utilizaremos la metodología de ML, y veremos si el algoritmo la ha aprendido.  

In [91]:
ruido = np.random.uniform(-1, 1, (observaciones,1))

targets = 2 * x1 - 3 * x2 + 5 + ruido

# Veamos las dimensiones. Deben ser n x m, donde m es el número de variables de salida.
print (targets.shape)

(1000, 1)


### Graficar los datos a usar para el entrenamiento

La idea es ver que haya una fuerte tendencia que nuestro modelo debe aprender a reproducir.


In [92]:
print(x1.shape)
print(x2.shape)
print(targets.shape)

(1000, 1)
(1000, 1)
(1000, 1)


In [82]:
x1N = x1.reshape(observaciones,)
x2N = x2.reshape(observaciones,)
targetsN = targets.reshape(observaciones,)

fig = px.scatter_3d(x = x1N, y = x2N, z = targetsN)

fig.update_layout(
    width = 500,
    height = 500,)

fig.show()

## Inicializar variables

Inicializaremos los pesos y sesgos, al azar, dentre de un rango inicial pequeño.  Es posible "jugar" con este valor pero no es recomendable ya que el uso de rangos iniciales altos inhibe el aprendizaje por parte del algoritmo

Los pesos son de dimensiones k x m, donde k es el numero de variables de entrada y m es el número de variables de salida.  

Como solo hay una salida, el sesgo es de tamaño 1, y es un escalar

In [83]:
rango_inicial = 0.1

weights = np.random.uniform(low = -rango_inicial, high = rango_inicial, size=(2, 1))

biases = np.random.uniform(low = -rango_inicial, high = rango_inicial, size=1)

#Veamos cómo fueron inicializados.
print (weights)
print (biases)

[[ 0.06108774]
 [-0.06445081]]
[-0.05907731]


In [84]:
weights.shape

(2, 1)

### Asignar la tasa de aprendizaje (Eta)

Se asigna un a tasa de aprendizaje pequeña.  Para este ejemplo funciona bien 0.02.  Vale la pena "jugar" con este valor para ver los resultados de hacerlo.

In [85]:
eta = 0.1

# Entrenar el modelo

Usaremos un valor de 100 para iterar sobre el conjunto de datos de entrenamiento.  Ese valor funciona bastante bien con la tasa de aprendizaje de 0.02.  Cómo saber el número adecuado de iteraciones es algo que veremos en futuras sesiones, pero generalmente una tasa de aprendizaje baja requiere de más iteraciones que una más alta.  Sin embargo hay que tener en mente que una tasa de aprendizaje alta puede causar que la pérdida "Loss" diverja a infinito, en vez de converger a cero (0)

Usaremos la función de pérdida L2-norm, pero dividido por 2, Para se consistente con la clase.  Es más, también lo dividiremos por el número de observaciones para obtener un promedio de pérdida por observación.  Hablamos en clase sobre la posiilidad de modificar esta función una vez no se pierda la característica de ser más baja para los resultados mejores, y vice versa.

Imprimimos la función de pérdida (loss) en cada iteración, para ver si está decreciendo como se desea.

Otro pequeño truco es escalar las deltas de la misma manera que se hizo con la función de pérdida.  De esta forma la tasa de aprendizaje es independiente del número de muestras (samples u observaciones).  De nuevo esto no cambia el principio, solo hace más fácil la selección de una tasa única de aprendizaje. 

Finalmente aplicamos la regla de actualización del decenso de gradiente.

Ojo!  los pesos son 2x1, la tasa de aprendizaje es 1x1 (escalar), las entradas son 1000x2, y las deltas escaladas son 1000x1.  Necesitamos obtener la transpuesta de las entradas para que no hayan problemas de dimensión en las operaciones. 



In [86]:
for i in range (100):
    
    # Esta es la ecuacion del modelo lineal: y = xw + b 
    salidas = np.dot(entradas, weights) + biases
    
    # Las deltas son las diferencias entre las salidas y las metas (targets)
    # deltas es un vector 1000 x 1
    deltas = salidas - targets
        
    loss = np.sum(deltas ** 2) / 2 / observaciones
    
    print(loss)
    
    deltas_escaladas = deltas / observaciones
      
    weights = weights - eta * np.dot(entradas.T, deltas_escaladas)
    biases = biases - eta * np.sum(deltas_escaladas)
    
    # Los pesos son actualizados en una forma de algebra lineal(una matriz menos otra)
    # Sin embargo, los sesgos en este caso son solo un número, es necesario transformar las deltas
    #       a un escalar.      
    # Ambas lineas son consistentes con la metodología de decenso de gradiente-

204.98318633785064
870.6664339330323
3926.2227027199506
18160.70658744326
85587.69090119601
410232.3447308968
1997216.7430108346
9861651.935405353
49303125.30920437
249139353.22820768
1270334242.645171
6525649930.858605
33725610174.573532
175150630991.6911
913164373853.2467
4775492634486.732
25034190523627.977
131482088490810.72
691570216667400.2
3641666726665858.0
1.9193178325943624e+16
1.0122540826025222e+17
5.3414682979240416e+17
2.8197329711311345e+18
1.4889876477539994e+19
7.864638689108492e+19
4.154769589875434e+20
2.1952149080132978e+21
1.1599914281500288e+22
6.130122476158026e+22
3.2397514428026813e+23
1.7122843553314316e+24
9.05016991718273e+24
4.783550622152578e+25
2.5284469794320087e+26
1.3364874577061446e+27
7.064504951023941e+27
3.734247039633838e+28
1.973912005651919e+29
1.0434104184150133e+30
5.515496023710485e+30
2.9155167465361492e+31
1.5411599046267882e+32
8.14668235018067e+32
4.3064020245584046e+33
2.2764017050520937e+34
1.2033269708692572e+35
6.360902230680396e+35
3

# Desplegamos los pesos y sesgos para ver si funcionaron correctamente.

Por el diseño de nuestro datos, los pesos debieran ser 2 y -3, y el sesgo: 5

**NOTA:**  Si aún no están los valores correctos, puede que aún estén convergiendo y sea necesario iterar más veces.  Para esto solo se requiere ejecutar la celda anterior cuantas veces sea requerido

In [87]:
print(weights, biases)   

[[-2.85843076e+36]
 [ 4.36666341e+34]] [-4.35767498e+33]


### Graficar las últimas salidas vrs las metas (targets)

Como son las últimas, luego del entrenamiento, representan el modelo final de exactitud.  Enntre más cercana esté esta gráfica a una línea de 45 grados, mhás cercanas están las salidas y metas.

Como este ejemplo es pequeño, es posible hacerlo, en los problemas que veremos más tarde en la clase, esto ya no sería posible.

In [88]:
salidasN = salidas.reshape(observaciones,)
targetsN = targets.reshape(observaciones,)
fig = px.scatter(x = salidasN, y =  targetsN)

fig.update_layout(
    width = 400,
    height = 400,)

fig.show()

## Preguntas del laboratorio

### 1) Cambie el número de observaciones a 100,000. Explique que es lo que ocurre en términos de:

Para responder las preguntas de este inciso se modifico el valor 1,000 del ejemplo visto en clase por 100,000
#### 1. El tiempo de ejecución para resolver el problemas
- Sin duda alguna el tiempo de ejecucion se ve afectado al utilizar un valor 100 veces mas grande. El algortimo cuando utilizo el valor de 1000 su resultado fue casi instantaneo, mientras que al utilizar 100000 llega a tardarse alrededor de 4 o 5 segundos. Adicionalemente cabe destacar que el desplege de las graficas tambien es mas tardado.

#### 2. El resultado final vrs lo encontrado en clase:  es igual, o diferente...¿porqué?
- En terminos de pesos podemos ver que al utilizar 100000 datos este cambia pero a cifras no muy significativas, mientras que con 1000 datos el sesgo era de 1.99746458 y -2.99849213, ahora los pesos son  1.99969807 y -2.99967002 esto representa que el nuevo modelo es ligeramente mejor que el anterior.
- El sesgo se mantiene casi identico con un valor de 4.9694638, mientras que el anterior era de 4.969634.
- Por las graficas tambien se puede notar que las salidas y las metas estan bastante cercanas ya que vemos una linea vertical de 45 grados bastante populada.

#### 3. Las graficas  para representar los datos/resultados
- Las graficas con las que se presentan los resultados son basicamente iguales, exeptuando que por la cantidad de datos la nueva se me mas llena. Ambas graficas representan una linea vertical con 45 grados.

### 2) Cambie el número de observaciones a 1,000,000.  Explique que es lo que ocurre en términos de:
Un caso muy similar al ejercicio anterior solo que ahora con 1000 veces el valor inicial utilizado en clase. El unico cambio realizado para responder estas preguntas fue cambiar el valor de la variable observaciones, de 1,000 a 1,000,000.

#### 1. El tiempo de ejecución para resolver el problemas
- En este caso ya es mucho mas evidente el cambio en el tiempo de ejecucion para resolver el problema, sin embargo, gran parte de este tiempo extra se ve reflejado a la hora de realizar y mostrar las graficas. En realidad, aun que el tiempo para realizar el entrenamiento tambien se vio afectado en 1 o 2 segundos, no es tan significativo como el tiempo utilizado para reproducir las graficas.

#### 2. El resultado final vrs lo encontrado en clase:  es igual, o diferente...¿porqué?
Primero que nada vemos que los resultados van mejorando pero muy poco.
- Los pesos cambian de de 1.99746458 y -2.99849213 a  2.00004472 y -2.99996543. Podemos notar que el ultimo valor es muy cercano a -3, sin embargo, el primero se pasa por muy poquito el valor de 2.
- El sesgo en este caso tampoco cambia significativamente.
- Por las graficas tambien se puede notar que las salidas y las metas estan bastante cercanas ya que vemos una linea vertical de 45 grados mucho mas populada que en los anteriores casos.

#### 3. Las graficas  para representar los datos/resultados
- La grafica que representa los datos en este caso tuvo inconvenientes al querer ser mostrada, primero que nada tardaba mucho en realizarse y mostrarse, y ya cuando se mostraba solo aparecia el plano sin ningun punto representativo.
- La grafica de resultados es casi identica a la de los otros casos, sigue presentandose una linea de 45 grados, sin embargo aqui es mas robusta.

### 3) “Juegue” un poco con el valor de la tasa de aprendizaje, por ejemplo  0.0001, 0.001, 0.1, 1.  Para cada uno de estos indique:
Para responder las preguntas de este inciso se regreso al numero incial de observaciones (1000), y lo unico que se cambio fue el valor de la taza de aprendizaje el utilizado en clase fue de 0.02, aqui se probo con los valores de 0.001, 0.1 y 1.
#### 1. ¿Qué ocurre con el tiempo de ejecución?
#### 2. ¿Qué ocurre con la minimización de la pérdida?
#### 3. ¿Qué ocurre con los pesos y los sesgos?
#### 4. ¿Qué ocurre con las iteraciones?
#### 5. ¿El problema queda resuelto o no?
#### 6. ¿Cuál es la apariencia de la última gráfica?¿Se cumple con la condición de que sea de 45 grados?

Acontinuacion se muestran las respuestas para cada valor probado:

#### 0.001:
1. En tiempo de ejeccucion no se ve ningun cambio significativo.
2. Vemos que con este valor la minimizacion de la perdida con 100 iteraciones no es suficiente para converger en 0. Son necesarias mas iteraciones.
3. Los pesos y sesgos tambien son afectados, en este caso ya no tenemos valores tan cercanos a 2 y -3 de los pesos y a nivel de sesgo si vemos un gran cambio al valor esperado 5. Aqui ya vemos una diferencia mas grande a nivel de decimales en estos valores. Valores obtenidos, pesos: 1.98 y -2.9, sesgo: 0.427
4. Observando la minimización de la perdida podemos concluir que para este valor son necesarias mas iteraciones.
5. No podemos decir que en efecto el problema sea resuelto ya que la grafica como tal no es una linea de 45 grados. Aunque la grafica aun representa una linea.
6. La grafica no es de 45 grados, vemos que en los datos de salida el valor maximo al que se alcanza es 50, mientras que en las metas es de 60.

#### 0.1:
1. Tampoco se ve un cambio significativo en el tiempo de ejecucion.
2. Para este valor la verdidad en lugar de converger a 0 este diverje hacia el infinito.
3. El cambio en pesos y sesgos con este valor tambien ya es mucho mas evidente, con valores demaisado alejados de los esperados. Pesos: -2.85843076e+36 y 4.36666341e+34, sesgo:-4.35767498e+33
4. El numero de iteraciones que coloquemos para este valor no importa, ya que igualmente divergera al infinito.
5. El problema no queda resuelto ya que al comprar los datos de salida con las metas nos damos cuenta que no tienen ningun tipo de relacion.
6. La grafica generada de los resultados es completamente inadecuada, primero que nada podemos notar una formta tipo rombo en lugar de una linea, tambien los valores de salida ocilan entre -1x10^34 y 1x10^37.

#### 1:
1.

### 4)Cambie la función de pérdida “L2-norm” a la misma pero sin dividir por 2.  Explique lo que ocurre en términos de:
#### 1.El tiempo que se tarda el algoritmo en terminar, comparado a lo que vimos en clase
#### 2.Si la pérdida se minimiza igual que lo que vimos en clase
#### 3.Si los pesos y sesgos son parecidos a los vistos en clase
#### 4.Si el problema se resuelve como ocurrió en clase
#### 5.Si se obtiene un mejor resultado al hacer más iteraciones

### 5)Cambie la función de pérdida de la “L2-norm” a la “L1-norm”.  Explique lo que ocurre en términos de:
#### 1.El tiempo que se tarda el algoritmo en terminar, comparado a lo que vimos en clase
#### 2.Si la pérdida se minimiza igual que lo que vimos en clase
#### 3.Si los pesos y sesgos son parecidos a los vistos en clase
#### 4.Si el problema se resuelve como ocurrió en clase
#### 5.Si se obtiene un mejor resultado al hacer más iteraciones
#### 6. ¿Tendrá una de estas más limitaciones que la otra?

### 6)Cree una función f(x,z) = 13 * xs + 7 * zs  - 12.
#### 1. ¿Funciona el algoritmo de la misma forma?